In [2]:
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [3]:
datatype = "train"

In [19]:
if datatype == "train": 
    in_ad = pd.read_csv("/home/hassan/lily/MLA/FDA/in_ad.csv", index_col=0)
    death = pd.read_csv("/home/bhatti/dataset/VCHAMPS/death_train.csv", index_col=0)

In [7]:
in_ad["Died during admission"]

5         0
8         0
9         0
11        0
15        0
         ..
644619    0
646569    0
653693    0
655923    0
656612    0
Name: Died during admission, Length: 522740, dtype: int64

In [5]:
in_ad.columns

Index(['Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Admitting unit service', 'Discharging unit service',
       'Admitting specialty', 'Discharging specialty',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory',
       'Discharge disposition', 'Died during admission',
       'Outpatientreferralflag', 'Serviceconnectedflag', 'State', 'Transfer',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'stay_length', 'Age 20-40', 'Age 40-60', 'Age 60-80',
       'Age 80-100', 'Age 100-120', 'cd_diagnosis', 'Mental', 'Medical',
       'Rehab', 'Gerontology', 'Others_Specialty', 'Agentorangeflag'],
      dtype='object')

In [14]:
def final_set_before(ids, group):
    
    readmission = 0
    num_admissions = group["Age at admission"].nunique()
    if num_admissions > 1: readmission=1

    age_mean = group["Age at admission"].mean()
    age_std = group["Age at admission"].std()
    if group["Age at admission"].nunique() == 1: age_std = 0

    min_age = group["Age at admission"].min()
    max_age = group["Age at admission"].max()

    freq = num_admissions/(math.floor(max_age - min_age) + 1)

    min_stay = group["stay_length"].min()
    max_stay = group["stay_length"].min()
    stay_mean = group["stay_length"].mean()
    stay_std = group["stay_length"].std()
    if group["stay_length"].nunique() == 1: stay_std = 0

    # num_cvd_readmission = max(0, group['cd_diagnosis'].sum() - 1)
    num_cvd_admission = group['cd_diagnosis'].sum()

    cvd = 0
    if group["cd_diagnosis"].sum() > 0: cvd = 1
    
    Died = 0
    if group["Died during admission"].sum() > 0: Died = 1

    AO = 0
    if group["Agentorangeflag"].sum() > 0: AO = 1

    if (group.iloc[len(group)-1]["Died during admission"] == 1) and (group.iloc[len(group)-1]['cd_diagnosis'] == 1): died_by_cvd = 1
    else: died_by_cvd = 0

    df = pd.DataFrame(data={'Internalpatientid': [ids], 'num_stays': [num_admissions], 'stay_length': group["stay_length"].sum(),
                            'num_unique_units': group["Admitting unit service"].nunique(), "num_transfers": group["Transfer"].sum(), 
                            "num_cvd_admission": [num_cvd_admission], "Died": [Died], "AO": [AO], "CVD": [cvd],
                            "unique_admitting_specialty": group["Admitting specialty"].nunique(), "unique_discharging_specialty": group["Discharging specialty"].nunique(),
                            "DOMICILIARY": group["DOMICILIARY"].sum(), "MEDICINE": group["MEDICINE"].sum(), "NHCU":group["NHCU"].sum(),
                            "NON-COUNT":group["NON-COUNT"].sum(), "OTHERS":group["OTHERS"].sum(), 'PSYCHIATRY': group['PSYCHIATRY'].sum(), 'SURGERY': group['SURGERY'].sum(),
                            'Age 20-40': group["Age 20-40"].sum(), 'Age 40-60': group["Age 40-60"].sum(), 'Age 60-80':group["Age 60-80"].sum(), 
                            'Age 80-100':group["Age 80-100"].sum(), 'Age 100-120':group["Age 100-120"].sum(), "age_mean": [age_mean], "age_std": [age_std], 
                            "age_min": [min_age], "age_max": [max_age], "stay_min": [min_stay], "stay_max": [max_stay], "stay_mean": [stay_mean],
                            "stay_std": [stay_std], "freq": [round(freq,2)], 'Medical': group["Medical"].sum(), 'Mental':group["Mental"].sum(), 
                            'Others_Specialty':group["Others_Specialty"].sum(), 'Rehab': group["Rehab"].sum(), 'Gerontology': group["Gerontology"].sum(),
                            "died_by_cvd": died_by_cvd
                            })
    df = df.reset_index(drop=True)
    return df

In [26]:
tidy_dataset = []
for ids, group in tqdm(in_ad.groupby("Internalpatientid")):
    group = group.sort_values(by = ['Admission date'],ascending=True).reset_index(drop = True)
    df = final_set_before(ids, group)
    tidy_dataset.append(df)
    
tidy_dataset = pd.concat(tidy_dataset)

  0%|          | 0/84536 [00:00<?, ?it/s]

In [27]:
tidy_dataset = tidy_dataset.merge(death, how="left",on="Internalpatientid")

In [28]:
# we only want dead patients
tidy_dataset["Died"] = [1] * len(tidy_dataset)
tidy_dataset.loc[tidy_dataset["Age at death"].isna(), "Died"] = 0
tidy_dataset = tidy_dataset[tidy_dataset["Died"] == 1]

In [33]:
tidy_dataset = tidy_dataset.drop(["Died", "Death date"], axis=1)

In [34]:
tidy_dataset

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,AO,CVD,unique_admitting_specialty,unique_discharging_specialty,...,stay_mean,stay_std,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,died_by_cvd,Age at death
2,3,2,407.50,2,0,1,0,1,2,2,...,203.750000,285.558003,0.67,0,1,0,1,0,0,82.284691
3,4,1,7.15,1,0,1,0,1,1,1,...,7.150000,0.000000,1.00,0,0,0,1,0,0,84.190359
4,5,2,8.45,2,0,1,0,1,2,1,...,4.225000,4.504270,2.00,2,0,0,0,0,0,76.506124
6,10,1,0.33,1,0,0,1,0,1,1,...,0.330000,0.000000,1.00,0,1,0,0,0,0,72.384665
9,15,4,127.10,3,0,3,1,1,4,3,...,31.775000,55.299774,0.21,2,0,1,1,0,0,86.598848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84530,169053,12,135.47,3,0,3,0,1,6,6,...,11.289167,17.461613,2.40,4,0,0,7,0,0,81.349588
84531,169055,1,6.47,1,0,1,0,1,1,1,...,6.470000,0.000000,1.00,0,1,0,0,0,0,68.572433
84533,169060,8,45.38,2,0,6,0,1,5,2,...,5.672500,7.616855,0.53,5,1,0,1,1,0,71.832486
84534,169062,12,178.63,4,1,0,1,0,7,5,...,14.885833,16.735598,3.00,10,2,0,0,0,0,74.181399


In [ ]:
if datatype == "train": tidy_dataset.to_csv("/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality.csv")